In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Valuation Debugging

Attributes
----------
valuation
transactions
instruments
recipes
valuation manifest
"""

toggle_code("Hide docstring")

# Valuation Analysis


## Table of contents

- 1. [Overview](#1.-Overview)
- 2. [Setup](#2.-Setup)
- 3. [Load Data](#3.-Load-Data)
   * [3.1 Portfolios](#3.1-Portfolios)
   * [3.2 Instruments](#3.2-Instruments)
   * [3.3 Transactions](#3.3-Transactions)   
   * [3.4 Quotes](#3.4-Quotes)
- 4. [Accrual Overrides](#4.-Accrual-Overrides)
   * [4.1 Match instruments with LUIDs](#4.1-Match-Instruments-With-LUID)
   * [4.2 Create Data Map](#4.2-Create-Data-Map)
- 5. [Valuations](#5.-Valuations)
   * [5.1 Valuation Recipe](#5.1-Valuation-Recipes)
   * [5.2 Valuation Function](#5.2-Valuation-Function)
   * [5.3 Valuation Analysis](#5.3-Valuation-Analysis)

## 1. Overview

One of the key constructs in LUSID is that of a Recipe. Recipes are a set of instructions for the valuation engine to determine how pricing will be conducted as well as what data will be used in the process. With Recipes, we can define things like which pricing sources to use (including fallback sources), which pricing models to use for various instrument types and what sort of lookback window to apply for quotes if a given quote doesn't exist on our valuation date.

Given the flexibility that Recipes provide around sourcing of market data, one of the things often required is the ability to see what is actually used during a valuation. For example, if a large quote lookback window is specified of say 10 days, one may wish to see if a quote was used that's older than a week. Additionally, if multiple quote sources are specified with a given precidence order, a user may want to see which source was used if the primary source had no quote available. Finally, for fixed income instruments a number of metrics can be calculated or read from an outside source and users may wish to know where these figures come from.

In this Notebook, we'll look at several key valuation metrics that can help us gain relevant insights to address the above concerns. We'll look at things through the lense of an alternative asset portfolio: 'GlobalAlternatives' as well as a fixed income portfolio: 'GlobalCredit'. The GlobalAlternatives portfolio contains six positions in a variety of alternative investment funds across farmland, climate, infrastructure and private credit, whereas the GlobalCredit portfolio contains 5 fixed income positions with two different flavours of bond.

Importantly, our example uses two seperate pricing sources with several of the funds having intermitant quotes across the month of January. The portfolios look as follows:

![Title](img/AlternativeFundStructure.png)

## 2. Setup

We first initialize our various Python libraries, objects, and datasets required to construct our examples:

In [2]:
# Import LUSID specific packages
# These are the core LUSID packages for interacting with the API via Python
import lusid
import lusid.models as models
import json
import pytz
import uuid
import luminesce
import lumipy
import numpy as np
from datetime import datetime
from dateutil.rrule import rrule, DAILY
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from flatten_json import flatten
from fbnsdkutilities import ApiClientFactory
import backoff

import os
import pandas as pd
import math
import io

# Set pandas dataframe display formatting
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate API Factorys which are client side objects for interacting with LUSID and Luminesce APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

lumi_api_factory = ApiClientFactory(
    luminesce,
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

#Load LUSID API Components
portfolio_api = api_factory.build(lusid.api.PortfoliosApi)
properties_api = api_factory.build(lusid.api.PropertyDefinitionsApi)
transaction_portfolio_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
instruments_api = api_factory.build(lusid.api.InstrumentsApi)
quotes_api = api_factory.build(lusid.api.QuotesApi)
configuration_recipe_api = api_factory.build(lusid.api.ConfigurationRecipeApi)
system_configuration_api = api_factory.build(lusid.api.SystemConfigurationApi)
aggregration_api = api_factory.build(lusid.api.AggregationApi)
srs_api = api_factory.build(lusid.api.StructuredResultDataApi)
lumi_sql_exe_api = lumi_api_factory.build(luminesce.SqlExecutionApi)

# Set Global Scope
global_scope = "Valuation_Analysis_NB"

# Transaction Portfolios
global_alt_portfolio_code = "GlobalAlternatives"
global_credit_portfolio_code = "GlobalCredit"

# Load Requisite Data
transaction_data = pd.read_excel("data/valuation_analysis_data.xlsx", sheet_name="transactions")
price_data = pd.read_excel("data/valuation_analysis_data.xlsx", sheet_name="market_prices")
instrument_data = pd.read_excel("data/valuation_analysis_data.xlsx", sheet_name="instruments")
bond_accrual_data = pd.read_excel("data/valuation_analysis_data.xlsx", sheet_name="bond_accruals")

## 3. Load Data

The 'valuation_analysis_data.xlsx' data file contains the requisite instrument definitions, transactions, and market quotes used in our example. 

### 3.1 Portfolios 

We first start by constructing our alternative asset portfolio 'GlobalAlternatives' and our fixed income portfolio 'GlobalCredit':

In [3]:
def create_portfolio(portfolio_code):
    # Create our Transaction Portfolios
    try:
        transaction_portfolio_api.create_portfolio(
            scope=global_scope,
            create_transaction_portfolio_request=models.CreateTransactionPortfolioRequest(
                display_name=portfolio_code,
                code=portfolio_code,
                base_currency="USD",
                created="2021-12-31",
                instrument_scopes=[global_scope]
            ),
        )
        print("Portfolio: " + portfolio_code + " loaded!")

    except lusid.ApiException as e:
        if 'PortfolioWithIdAlreadyExists' not in json.loads(e.body)["name"]:
            print(json.loads(e.body)["title"])
        
create_portfolio(global_alt_portfolio_code)
create_portfolio(global_credit_portfolio_code)

### 3.2 Instruments

Here we will create the functions to create instruments for each instrument type in the data file.

#### 3.2.1 Create Equity Function

In [4]:
def create_equity(data):
          
    client_internal = "Instrument/default/ClientInternal"
    
    equity = models.Equity(
        instrument_type="Equity",
        dom_ccy=data["currency"],
    )

    equity_definition = models.InstrumentDefinition(
        name=data["name"],
        identifiers={"ClientInternal": models.InstrumentIdValue(data["client_internal"])},
        definition=equity,
        properties=[]      
    )

    # upsert the instrument
    upsert_request = {client_internal: equity_definition}
    upsert_response = instruments_api.upsert_instruments(scope=global_scope, request_body=upsert_request)
    equity_luid = upsert_response.values[client_internal].lusid_instrument_id
    return (equity_luid,data["client_internal"],data["currency"])

#### 3.2.2 Create Bond Function

In [5]:
def create_bond(data):

    flow_conventions = models.FlowConventions(
        currency=data["currency"],
        payment_frequency=data["payment_frequency"],
        roll_convention=data["roll_convention"],
        day_count_convention=data["day_count_convention"],
        payment_calendars=[],
        reset_calendars=[],
        settle_days=data["settle_days"],
        reset_days=data["reset_days"]
    )

    bond = models.Bond(
        start_date=data["start_date"].date(),
        maturity_date=data["maturity_date"].date(),
        dom_ccy=data["dom_ccy"],
        principal=data["principal"],
        coupon_rate=data["coupon_rate"],
        flow_conventions=flow_conventions,
        identifiers={},
        instrument_type="Bond",   
        calculation_type="Standard",
    )

    # define the instrument to be upserted
    bond_definition = models.InstrumentDefinition(
        name=data["bond_name"],
        identifiers={"ClientInternal": models.InstrumentIdValue(row["client_internal"])},
        definition=bond,
    )

    # upsert the instrument
    upsert_request = {row["bond_identifier"]: bond_definition}
    upsert_response = instruments_api.upsert_instruments(scope=global_scope,request_body=upsert_request)
    bond_luid = upsert_response.values[row["bond_identifier"]].lusid_instrument_id
    return (bond_luid,data["client_internal"],data["currency"])

#### 3.2.3 Create Simple Instrument Function

In [6]:
def create_simple_instrument(data):
          
    client_internal = "Instrument/default/ClientInternal"
    
    simple_instrument = models.SimpleInstrument(
        instrument_type="SimpleInstrument",
        dom_ccy=data["currency"],
        asset_class="Credit",
        simple_instrument_type=data["instrument_classification"]
    )

    simple_instrument_definition = models.InstrumentDefinition(
        name=data["name"],
        identifiers={"ClientInternal": models.InstrumentIdValue(data["client_internal"])},
        definition=simple_instrument,
        properties=[]      
    )

    # upsert the instrument
    upsert_request = {client_internal: simple_instrument_definition}
    upsert_response = instruments_api.upsert_instruments(scope=global_scope, request_body=upsert_request)
    simple_instrument_luid = upsert_response.values[client_internal].lusid_instrument_id
    return (simple_instrument_luid,data["client_internal"],data["currency"])

#### 3.2.4 Create instruments

Here we call our functions to create instruments. We will also need to record the internal LUSID instrument IDs in order to use them later in the SRS (information on the SRS can be found [here](https://support.lusid.com/knowledgebase/article/KA-01893/en-us)).

In [7]:
Luids = []

# Load instruments
for index, row in instrument_data.iterrows():
    
    if row["instrument_type"] == "Equity":
        Luids.append(create_equity(row))
    elif row["instrument_type"] == "Bond":
        Luids.append(create_bond(row))
    else:
        Luids.append(create_simple_instrument(row))
        
print ("Instruments Upserted!")

LuidMap = pd.DataFrame(Luids,columns = ['LusidInstrumentId','client_internal','Currency'])

LuidMap

Instruments Upserted!


,LusidInstrumentId,client_internal,Currency
0,LUID_00003DS5,cid_fund_farmland,USD
1,LUID_00003DS6,cid_fund_carboncredit,USD
2,LUID_00003DS7,cid_fund_euroinfra,EUR
3,LUID_00003DS8,cid_fund_asianinfra,USD
4,LUID_00003DS9,cid_fund_privatecreditopp,USD
5,LUID_00003DSA,cid_fund_globaldistressedcredit,USD
6,LUID_00003DSB,cid_glbcd_abccorp,USD
7,LUID_00003DSC,cid_glbcd_xyzglobal,USD
8,LUID_00003DSD,cid_glbcd_highllp,USD
9,LUID_00003DSE,cid_glbcd_housingr,USD


### 3.3 Transactions
To construct our holdings, we load in a set of transaction data across our 2 portfolios.

In [8]:
# Load transactions
for index, row in transaction_data.iterrows():

    primary_instrument_identifier = { "Instrument/default/ClientInternal": row["client_internal"] }
    
    if isinstance(row["client_internal"], float):
        primary_instrument_identifier = { "Instrument/default/Currency": row["currency"] }

    upsert_transactions = transaction_portfolio_api.upsert_transactions(
        scope=global_scope,
        code=row["portfolio"],
        transaction_request=[
            models.TransactionRequest(
                transaction_id=row["txn_id"],
                type=row["txn_type"],
                instrument_identifiers=primary_instrument_identifier,
                transaction_date=row["trade_date"],
                settlement_date=row["settle_date"],
                units=row["quantity"],
                transaction_price=models.TransactionPrice(
                    price=row["txn_price"], type="Price"
                ),
                total_consideration=models.CurrencyAndAmount(
                    amount=row["total_consideration"], currency=row["currency"]
                ),
            )
        ],
    )

print ("Transactions Upserted!")

Transactions Upserted!


### 3.4 Quotes
We now load in the relevant market prices for our valuations.

In [9]:
#Load market prices
instrument_quotes = {
    str(uuid.uuid4()): models.UpsertQuoteRequest(
            quote_id=models.QuoteId(
                quote_series_id=models.QuoteSeriesId(
                    provider="LUSID",
                    instrument_id=price["id"],
                    instrument_id_type=price["id_type"],
                    quote_type="Price",
                    field="mid",
                    price_source=price["price_source"],
                ),
                effective_at=price["date"],
            ),
            metric_value=models.MetricValue(value=price["price"], unit=price["currency"]),
            scale_factor=price["scale_factor"]
    )
    for row, price in price_data.iterrows()
}

# Upsert the quotes into LUSID
response = quotes_api.upsert_quotes(
    scope=global_scope, request_body=instrument_quotes
)

print ("Quotes Upserted!")

Quotes Upserted!


## 4. Accrual Overrides

In our GlobalCredit portfolio we have two Instrument types: Corporate bonds and ABS. LUSID is able to calculate accrual values for corporate bonds, but can't fully model and calculate analytics for ABS instruments at present. Here we will upload some bond accrual data from our data file for the instruments which LUSID can't calculate. To do this, we need to create a map for the data and then upload it as a structured result store (SRS) document. Full details around the structured result data can be found in [this knowledge base article](https://support.lusid.com/knowledgebase/article/KA-01893/en-us)

### 4.1 Match Instruments With LUID

To upload the SRS data we need to provide the LUSID instrument ID (LUID) along with our accruals data. We can do this by using the LUID map we created when we upserted the instruments.

In [10]:
accrual_srs = pd.merge(bond_accrual_data,LuidMap, how="left", on="client_internal")
accrual_srs

,date,client_internal,accrued_interest,LusidInstrumentId,Currency
0,2022-01-03T00:00:00Z,cid_glbcd_housingr,0.00,LUID_00003DSE,USD
1,2022-01-04T00:00:00Z,cid_glbcd_housingr,0.00,LUID_00003DSE,USD
2,2022-01-05T00:00:00Z,cid_glbcd_housingr,0.00,LUID_00003DSE,USD
3,2022-01-06T00:00:00Z,cid_glbcd_housingr,0.00,LUID_00003DSE,USD
4,2022-01-07T00:00:00Z,cid_glbcd_housingr,0.01,LUID_00003DSE,USD
5,2022-01-10T00:00:00Z,cid_glbcd_housingr,0.01,LUID_00003DSE,USD
6,2022-01-11T00:00:00Z,cid_glbcd_housingr,0.01,LUID_00003DSE,USD
7,2022-01-12T00:00:00Z,cid_glbcd_housingr,0.01,LUID_00003DSE,USD
8,2022-01-13T00:00:00Z,cid_glbcd_housingr,0.01,LUID_00003DSE,USD
9,2022-01-14T00:00:00Z,cid_glbcd_housingr,0.01,LUID_00003DSE,USD


### 4.2 Create Data Map

Before uploading the SRS data we need a way to map each column to a valid SRS address, so that it can be used later.

In [11]:
data_map_key = models.DataMapKey(
    code = "sample-data-map",
    version = "1.0.16"
)    

def upsert_structured_result_data_map(data_map_key):
    
    try:
        srs_api.create_data_map(
            scope = global_scope,
            request_body = {
                "data-map": models.CreateDataMapRequest(
                    id=data_map_key,
                    data=models.DataMapping(
                        data_definitions=[
                            models.DataDefinition(address="UnitResult/LusidInstrumentId", name="LusidInstrumentId", data_type="String", key_type="Unique"),
                            models.DataDefinition(address="UnitResult/Valuation/InstrumentAccrued", data_type="Result0D", key_type="CompositeLeaf"),
                            models.DataDefinition(address="UnitResult/Valuation/InstrumentAccrued/Amount", name="accrued_interest", data_type="Decimal", key_type="Leaf"),
                            models.DataDefinition(address="UnitResult/Valuation/InstrumentAccrued/Ccy", name="Currency", data_type="String", key_type="Leaf"),
                        ]
                    )
                )
            }
        )
    except lusid.ApiException as e:
        if 'DataMap exists' not in json.loads(e.body)["detail"]:
            print("Error loading data map: " + json.loads(e.body)["detail"])
    
upsert_structured_result_data_map(data_map_key)

Now we upload SRS data for each effective date.

In [12]:
srs_ids = []

for effective_at, srs_df in accrual_srs.groupby("date"):
    
    srs_data_id = models.StructuredResultDataId(
        source="Client",
        code="BondAccrual",
        effective_at=effective_at,
        result_type = "UnitResult/Analytic"
    )
    
    srs_ids.append(srs_data_id)
    
    s = io.StringIO()
    srs_df.to_csv(s)
    
    srs_data = models.StructuredResultData(
        document_format="Csv",
        version="0.1.1",
        name="Bond Accrual",
        data_map_key=data_map_key,
        document=s.getvalue()        
    )
    
    srs_api.upsert_structured_result_data(
        scope=global_scope, 
        request_body={ 
            "data": models.UpsertStructuredResultDataRequest(
                id=srs_data_id, 
                data=srs_data
            )
        }
    )

# 5. Valuations

Now that we have our data uploaded, we will perform valuations and do some analysis on the figures returned. First we will define the functions needed.

## 5.1 Valuation Recipes

In order to perform our valuations we need to create valuation recipes. Here we will create 2 valuation recipes with some common attributes. Both recipes will:

- Use prices from the month_end_accounting price source first if available, with a quote interval of 10 days.
- Use prices from the market_vendor price source second, with a quote interval of 10 days.
- Use the SimpleStatic pricing model for bonds.
- Override calculated accrued interest values with ones in the SRS where available.

The recipes will differ in that the first, called "ValuationAnalysisRecipe" will not allow partially successful valuations whereas the second, called "ValuationAnalysisRecipeWPartialResult" will allow partially successful valuations. We will look at what that means later.

In [13]:
# Create two different recipes depending on the AllowPartiallySuccessfulEvaluation option
def UpsertRecipe(recipe_code, allow_partial_results):
    
    pricing_options = {}             
    
    if allow_partial_results == True:
        pricing_options={"AllowPartiallySuccessfulEvaluation": True}
    
    configuration_recipe = models.ConfigurationRecipe(
        scope=global_scope,
        code=recipe_code,
        market=models.MarketContext(
            market_rules=[
                models.MarketDataKeyRule(
                    key="Quote.ClientInternal.*",
                    supplier="Lusid",
                    data_scope=global_scope,
                    quote_type="Price",
                    field="mid",
                    quote_interval="10D",
                    price_source='month_end_accounting',
                ),
                models.MarketDataKeyRule(
                    key="Quote.ClientInternal.*",
                    supplier="Lusid",
                    data_scope=global_scope,
                    quote_type="Price",
                    field="mid",
                    quote_interval="10D",
                    price_source='market_vendor',
                ),                
            ],
            options=models.MarketOptions(
                default_supplier="Lusid",
                default_instrument_code_type="ClientInternal",
                default_scope=global_scope,
                attempt_to_infer_missing_fx=True             
            ),
        ),
        pricing=models.PricingContext(
            model_rules=[
                models.VendorModelRule(
                    supplier="Lusid",
                    model_name="SimpleStatic",
                    instrument_type="Bond",
                    parameters="{}",
                )
             ],             
            result_data_rules=[
                 models.ResultDataKeyRule(
                     resource_key="UnitResult/Valuation/InstrumentAccrued",
                     supplier="Client",
                     data_scope=global_scope,
                     document_code="BondAccrual",
                     quote_interval="1D",
                     document_result_type="UnitResult/Analytic",
                     result_key_rule_type="ResultDataKeyRule"
                 )
                ],
            options=pricing_options
        )
    )

    upsert_configuration_recipe_response = (
        configuration_recipe_api.upsert_configuration_recipe(
            upsert_recipe_request=models.UpsertRecipeRequest(
                configuration_recipe=configuration_recipe
            )
        )
    )
    
    print (f"Recipe {recipe_code} Upserted!")
    
UpsertRecipe("ValuationAnalysisRecipe", False)
UpsertRecipe("ValuationAnalysisRecipeWPartialResult", True)

Recipe ValuationAnalysisRecipe Upserted!
Recipe ValuationAnalysisRecipeWPartialResult Upserted!


## 5.2 Valuation Function

Now we create a function to perform the valuation and load the results into a data frame. 

In [14]:
def get_valuation(date, portfolio_code, recipe_code, metrics_dict):    
    try:
        metrics_list = []
        
        for m in metrics_dict.keys():
            metrics_list.append(models.AggregateSpec(m,"Value"))
        
        # Build and run valuation request
        valuation_request = models.ValuationRequest(
            recipe_id=models.ResourceId(scope=global_scope, code=recipe_code),
            metrics=metrics_list,
            portfolio_entity_ids=[
                models.PortfolioEntityId(scope=global_scope, code=portfolio_code)
            ],
            valuation_schedule=models.ValuationSchedule(effective_at=date),
        )

        val_response = aggregration_api.get_valuation(valuation_request=valuation_request)
        val_data = val_response.data
        vals_df = pd.DataFrame(val_data).rename(columns=metrics_dict)
        
        if "Aggregation/Errors" in vals_df.columns: vals_df.rename(columns={"Aggregation/Errors" : "Errors"}, inplace=True )
        
        return vals_df
    
    except lusid.ApiException as e:
        print(json.loads(e.body)["errorDetails"][0]["id"])

## 5.3 Valuation Analysis

### 5.3.1 Allow Partial Results

For our first valuation, we would like to look at the Global Alternatives portfolio. Firstly let's do a month end valuation for the 31st January:

In [15]:
metrics = { "Instrument/default/Name" : "Instrument name",
            "Valuation/PV" : "PV",
            "Quotes/Price/PriceSource" : "Price Source"}

get_valuation("2022-01-31T00:00:00Z", "GlobalAlternatives", "ValuationAnalysisRecipe", metrics)


,Instrument name,PV,Price Source
0,LUSID AM Farmland Fund - Class A,"201,000.00",month_end_accounting
1,USD,"14,579.50",None
2,LUSID AM Carbon Credit (CO2) Fund,"53,375.00",month_end_accounting
3,LUSID AM European Infrastructure Fund,"71,437.50",month_end_accounting
4,EUR,"3,750.00",None
5,LUSID AM Asian Infrastructure Fund,"271,050.00",month_end_accounting
6,LUSID AM Private Credit Opportunities Fund,"22,950.00",market_vendor
7,LUSID AM Global Distressed Credit Fund,"240,947.50",market_vendor


That one worked fine but now we want to perform a valuation for a different date:

In [16]:
get_valuation("2022-01-29T00:00:00Z", "GlobalAlternatives", "ValuationAnalysisRecipe", metrics)

Failed to resolve market data item [Provider: Lusid, PriceSource: month_end_accounting, InstrumentId: cid_fund_euroinfra, InstrumentIdType: ClientInternal, QuoteType: Price, Field: mid].


This valuation has run into an error and can't be performed. We would like to see results for the rows that did not return an error, so if we run it with our second recipe which allows a return with partial results we can see some figures:

In [17]:
get_valuation("2022-01-29T00:00:00Z", "GlobalAlternatives", "ValuationAnalysisRecipeWPartialResult", metrics)

,Instrument name,PV,Price Source,Errors
0,LUSID AM Farmland Fund - Class A,"203,500.00",market_vendor,[]
1,USD,"14,579.50",None,[]
2,LUSID AM Carbon Credit (CO2) Fund,"53,250.00",market_vendor,[]
3,LUSID AM European Infrastructure Fund,NaN,None,[One or more failures occurred. Failed to reso...
4,EUR,"3,750.00",None,[]
5,LUSID AM Asian Infrastructure Fund,"271,050.00",market_vendor,[]
6,LUSID AM Private Credit Opportunities Fund,"23,390.00",market_vendor,[]
7,LUSID AM Global Distressed Credit Fund,"241,535.00",market_vendor,[]


### 5.3.2 Quote Age

Now we might want to check how reliable/stale our quotes are. For that we can call on the effective date of the quote by adding that address to the metrics list:

In [18]:
metrics = { "Instrument/default/Name" : "Instrument name",
            "Valuation/PV" : "PV",
            "Quotes/Price/PriceSource" : "Price Source",
            "Quotes/Price/EffectiveAt" : "Price Date"}

get_valuation("2022-01-14T00:00:00Z", "GlobalAlternatives", "ValuationAnalysisRecipe", metrics)

,Instrument name,PV,Price Source,Price Date
0,LUSID AM Farmland Fund - Class A,"204,000.00",market_vendor,2022-01-10T00:00:00.0000000+00:00
1,USD,"14,579.50",None,None
2,LUSID AM Carbon Credit (CO2) Fund,"53,125.00",market_vendor,2022-01-13T00:00:00.0000000+00:00
3,LUSID AM European Infrastructure Fund,"71,400.00",market_vendor,2022-01-14T00:00:00.0000000+00:00
4,EUR,"3,750.00",None,None
5,LUSID AM Asian Infrastructure Fund,"267,150.00",market_vendor,2022-01-14T00:00:00.0000000+00:00
6,LUSID AM Private Credit Opportunities Fund,"24,140.00",market_vendor,2022-01-14T00:00:00.0000000+00:00
7,LUSID AM Global Distressed Credit Fund,"241,742.50",market_vendor,2022-01-14T00:00:00.0000000+00:00


If we want this in a more readable format, we can create a function to calculate the age of the quote compared to the given date

In [19]:
def parse_lusid_date(quote_date_str, valuation_date_str):
    if quote_date_str is None:
        return None
    else:        
        quote_date = datetime.strptime(quote_date_str.split('.')[0], "%Y-%m-%dT%H:%M:%S");
        valuation_date = datetime.strptime(valuation_date_str.split('.')[0], "%Y-%m-%dT%H:%M:%SZ");
        return valuation_date - quote_date

eff_date = "2022-01-14T00:00:00Z"
valuation = get_valuation(eff_date, "GlobalAlternatives", "ValuationAnalysisRecipe", metrics)

valuation['Quote Age (in days)'] = valuation["Price Date"].apply(parse_lusid_date, args=(eff_date,))

valuation

,Instrument name,PV,Price Source,Price Date,Quote Age (in days)
0,LUSID AM Farmland Fund - Class A,"204,000.00",market_vendor,2022-01-10T00:00:00.0000000+00:00,4 days
1,USD,"14,579.50",None,None,NaT
2,LUSID AM Carbon Credit (CO2) Fund,"53,125.00",market_vendor,2022-01-13T00:00:00.0000000+00:00,1 days
3,LUSID AM European Infrastructure Fund,"71,400.00",market_vendor,2022-01-14T00:00:00.0000000+00:00,0 days
4,EUR,"3,750.00",None,None,NaT
5,LUSID AM Asian Infrastructure Fund,"267,150.00",market_vendor,2022-01-14T00:00:00.0000000+00:00,0 days
6,LUSID AM Private Credit Opportunities Fund,"24,140.00",market_vendor,2022-01-14T00:00:00.0000000+00:00,0 days
7,LUSID AM Global Distressed Credit Fund,"241,742.50",market_vendor,2022-01-14T00:00:00.0000000+00:00,0 days


### 5.3.3 Valuation Manifest
Another tool which can be used to analyse valuations is the valuation manifest. The manifest is a body of information produced by each valuation that explains how the results of every call to the GetValuation API are generated. It can answer questions like ‘how were my market rules resolved?’ and ‘how many times did I access pricing data from a particular vendor for a particular instrument?’. Full details on the valuation manifest can be found in [this knowledge base article](https://support.lusid.com/knowledgebase/article/KA-01892/en-us)

To get the manifest, we need the request ID which gets returned when the valuation is run. We can create a new valuation function for that.

In [20]:
def get_valuation_with_req_id(date, portfolio_code, recipe_code, metrics_dict):
    
    try:
        metrics_list = []
        
        for m in metrics_dict.keys():
            metrics_list.append(models.AggregateSpec(m,"Value"))
            
        # Build and run valuation request
        valuation_request = models.ValuationRequest(
            recipe_id=models.ResourceId(scope=global_scope, code=recipe_code),
            metrics=metrics_list,
            portfolio_entity_ids=[
                models.PortfolioEntityId(scope=global_scope, code=portfolio_code)
            ],
            valuation_schedule=models.ValuationSchedule(effective_at=date),
        )

        val_response = aggregration_api.get_valuation(valuation_request=valuation_request)
        val_data = val_response.data
        vals_df = pd.DataFrame(val_data).rename(columns=metrics_dict)
        if "Aggregation/Errors" in vals_df.columns: vals_df.rename(columns={"Aggregation/Errors" : "Errors"}, inplace=True )
        resp_id = val_response.links[0].href[-22:]
        
        return (resp_id,vals_df)
    
    except lusid.ApiException as e:
        print(json.loads(e.body)["errorDetails"][0]["id"])


Now a function to retrieve the valuation manifest. The manifest must be retrieved using Luminesce, which can be accessed with the lumipy package.

In [21]:
# Function to request the valuations manifest given a specific request ID and effective date.
# The manifest is written asynchronously and there may be a lag for them to be queryable after performing the valuation.
# The backoff decorator on this function will cause it to retry until the manifest is retrieved (up to 5 retries).

@backoff.on_predicate(backoff.expo, lambda x: len(x.values) < 1, max_tries = 5)
def get_val_manifest(req_id, eff_date):

    res = lumi_sql_exe_api.put_by_query_csv(body=f"""
    --lumipy
    select * from Lusid.Logs.Valuations.Manifest 
    where UserRequestId = '{req_id}' and 
          EffectiveAt = #{eff_date}#   
    """,
    query_name="query",
    timeout_seconds=30)

    return pd.read_csv(io.StringIO(res), encoding='utf-8')

We can then perform a valuation and retrieve the manifest. The manifest shows us that price data was requested for each instrument for both the month_end_accounting and market_vendor price sources.

In [22]:
eff_date = "2022-01-14T00:00:00Z"

valuation = get_valuation_with_req_id(eff_date, "GlobalAlternatives", "ValuationAnalysisRecipeWPartialResult", metrics)

valuation[1]

,Instrument name,PV,Price Source,Price Date
0,LUSID AM Farmland Fund - Class A,"204,000.00",market_vendor,2022-01-10T00:00:00.0000000+00:00
1,USD,"14,579.50",None,None
2,LUSID AM Carbon Credit (CO2) Fund,"53,125.00",market_vendor,2022-01-13T00:00:00.0000000+00:00
3,LUSID AM European Infrastructure Fund,"71,400.00",market_vendor,2022-01-14T00:00:00.0000000+00:00
4,EUR,"3,750.00",None,None
5,LUSID AM Asian Infrastructure Fund,"267,150.00",market_vendor,2022-01-14T00:00:00.0000000+00:00
6,LUSID AM Private Credit Opportunities Fund,"24,140.00",market_vendor,2022-01-14T00:00:00.0000000+00:00
7,LUSID AM Global Distressed Credit Fund,"241,742.50",market_vendor,2022-01-14T00:00:00.0000000+00:00


In [23]:
get_val_manifest(valuation[0],eff_date)

,UserRequestId,EconomicDependency,AsAt,MarketDataKey,Mask,Supplier,PriceSource,DataScope,QuoteInterval,QuoteType,QuoteInstrumentType,Field,JsonMarketDataObject
0,0HMOVP1JNF6FO:00000019,"{\n ""DependencyType"": ""Opaque""\n}",2023-03-08 11:53:16.042,Quote.ClientInternal.*,NaN,Lusid,month_end_accounting,Valuation_Analysis_NB,10D,Price,ClientInternal,mid,"{\n ""Failure"": ""Failed to resolve market data..."
1,0HMOVP1JNF6FO:00000019,"{\n ""DependencyType"": ""Opaque""\n}",2023-03-08 11:53:16.042,Quote.ClientInternal.*,NaN,Lusid,market_vendor,Valuation_Analysis_NB,10D,Price,ClientInternal,mid,"{\n ""QuoteId"": {\n ""QuoteSeriesId"": {\n ..."
2,0HMOVP1JNF6FO:00000019,"{\n ""DependencyType"": ""Opaque""\n}",2023-03-08 11:53:16.042,Quote.ClientInternal.*,NaN,Lusid,month_end_accounting,Valuation_Analysis_NB,10D,Price,ClientInternal,mid,"{\n ""Failure"": ""Failed to resolve market data..."
3,0HMOVP1JNF6FO:00000019,"{\n ""DependencyType"": ""Opaque""\n}",2023-03-08 11:53:16.042,Quote.ClientInternal.*,NaN,Lusid,market_vendor,Valuation_Analysis_NB,10D,Price,ClientInternal,mid,"{\n ""QuoteId"": {\n ""QuoteSeriesId"": {\n ..."
4,0HMOVP1JNF6FO:00000019,"{\n ""DependencyType"": ""Opaque""\n}",2023-03-08 11:53:16.042,Quote.ClientInternal.*,NaN,Lusid,month_end_accounting,Valuation_Analysis_NB,10D,Price,ClientInternal,mid,"{\n ""Failure"": ""Failed to resolve market data..."
5,0HMOVP1JNF6FO:00000019,"{\n ""DependencyType"": ""Opaque""\n}",2023-03-08 11:53:16.042,Quote.ClientInternal.*,NaN,Lusid,market_vendor,Valuation_Analysis_NB,10D,Price,ClientInternal,mid,"{\n ""QuoteId"": {\n ""QuoteSeriesId"": {\n ..."
6,0HMOVP1JNF6FO:00000019,"{\n ""DependencyType"": ""Opaque""\n}",2023-03-08 11:53:16.042,Quote.ClientInternal.*,NaN,Lusid,month_end_accounting,Valuation_Analysis_NB,10D,Price,ClientInternal,mid,"{\n ""Failure"": ""Failed to resolve market data..."
7,0HMOVP1JNF6FO:00000019,"{\n ""DependencyType"": ""Opaque""\n}",2023-03-08 11:53:16.042,Quote.ClientInternal.*,NaN,Lusid,market_vendor,Valuation_Analysis_NB,10D,Price,ClientInternal,mid,"{\n ""QuoteId"": {\n ""QuoteSeriesId"": {\n ..."
8,0HMOVP1JNF6FO:00000019,"{\n ""DependencyType"": ""Opaque""\n}",2023-03-08 11:53:16.042,Quote.ClientInternal.*,NaN,Lusid,month_end_accounting,Valuation_Analysis_NB,10D,Price,ClientInternal,mid,"{\n ""Failure"": ""Failed to resolve market data..."
9,0HMOVP1JNF6FO:00000019,"{\n ""DependencyType"": ""Opaque""\n}",2023-03-08 11:53:16.042,Quote.ClientInternal.*,NaN,Lusid,market_vendor,Valuation_Analysis_NB,10D,Price,ClientInternal,mid,"{\n ""QuoteId"": {\n ""QuoteSeriesId"": {\n ..."


If we pick one of the errors returned and drill down into the JsonMarketDataObject row, we can see that a quote was not found for that price source and date.

In [24]:
get_val_manifest(valuation[0],eff_date)["JsonMarketDataObject"][0]

'{\n  "Failure": "Failed to resolve market data item [Provider: Lusid, PriceSource: month_end_accounting, InstrumentId: cid_fund_globaldistressedcredit, InstrumentIdType: ClientInternal, QuoteType: Price, Field: mid]. Attempted to resolve with supplier [Lusid] and sourceSystem [Lusid] in scope [Valuation_Analysis_NB] for effective date [14/01/2022 00:00:00], quoteInterval [10D] and with predicate [AsAt=Latest], but failed because failed to find quote in store."\n}'

### 5.3.4 Calculated vs SRS Data

For the bonds in our GlobalCredit portfolio, we would like to display the accrued interest in the valuation. For the 3 corporate bond positions this can be calculated by LUSID however for the 2 ABS positions we will need to use the data we loaded into the SRS in section 4.

In [25]:
metrics = { "Instrument/default/Name" : "Instrument name",
            "Valuation/PV" : "PV",
            "Valuation/Accrued" : "Accrued"}

get_valuation("2022-01-28T00:00:00Z", "GlobalCredit", "ValuationAnalysisRecipeWPartialResult", metrics)

,Instrument name,PV,Accrued
0,AC 5.5 02/15/2030,"99,432.82","2,480.98"
1,USD,"235,400.00",0.00
2,XYZ 4.75 01/03/2023,"196,561.97","3,910.22"
3,HT 6.25 20/06/2025,"147,288.53","1,015.62"
4,Housing LLC C1R 6.250%,"49,226.99","1,000.00"
5,Aircraft LLC E1 7.000%,"59,229.26","1,290.00"


If we want to know whether the accrued value came from the SRS or not, we can explicitly request the value using the "UnitResult" address. If a value is returned then the SRS has been used, if not then it has been calculated.

In [26]:
metrics = [
    models.AggregateSpec("Instrument/default/Name", "Value"),
    models.AggregateSpec("Valuation/PV", "Value"),
    models.AggregateSpec("Valuation/Accrued", "Value"),
    models.AggregateSpec("UnitResult/Valuation/InstrumentAccrued", "Value")
]

metrics = { "Instrument/default/Name" : "Instrument name",
            "Valuation/PV" : "PV",
            "Valuation/Accrued" : "Accrued",
            "UnitResult/Valuation/InstrumentAccrued" : "SRS Accrued"}

valuation = get_valuation("2022-01-28T00:00:00Z", "GlobalCredit", "ValuationAnalysisRecipeWPartialResult", metrics)

valuation["Accrual Source"] = np.where(valuation["SRS Accrued"].isnull(),"Calculation","SRS")

del valuation["SRS Accrued"]
del valuation["Errors"]

valuation

,Instrument name,PV,Accrued,Accrual Source
0,AC 5.5 02/15/2030,"99,432.82","2,480.98",Calculation
1,USD,"235,400.00",0.00,Calculation
2,XYZ 4.75 01/03/2023,"196,561.97","3,910.22",Calculation
3,HT 6.25 20/06/2025,"147,288.53","1,015.62",Calculation
4,Housing LLC C1R 6.250%,"49,226.99","1,000.00",SRS
5,Aircraft LLC E1 7.000%,"59,229.26","1,290.00",SRS
